## Premise & Input
- Elves have food management system we need to parse to count how many ingredients are still fresh
- Input consists of two parts:
  1. fresh ID ranges "3-5" = [3,5] 
  2. ingredient IDs 

- example input: 

```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```

- read in fresh ranges to array, unpack ranges into a set
- for each ingredient ID being read in, if it is in the set, increment fresh count, else continue


In [ ]:
infile = open("input.txt", "r")
ranges = []
validIDs = set()
line = infile.readline().strip()
while line != "": # use empty line between ranges and ingredients sections
    print(line)
    bounds = line.split("-")
    #validIDs = validIDs.union(set(range(int(bounds[0]), int(bounds[1])+1))) # I like this logic, but it is very inefficient
    line = infile.readline().strip()



## Time to pivot!
- the fresh ID ranges are very large numbers, so converting all the numbers between the bounds to set elements takes longer on a single range than we would like the entire script to take
- instead of generating a set, treat the bounds as bounds for comparison checks
- also preprocessing ingredients ID into an ascending list is cheap and helpful for algorithm
  
  
Prepare input
- read in ranges into 2d list, [[start,end],[start,end],...]
- sort ID ranges by ascending start value
- read in ingredients ID to list
- sort ingredients ID ascending 


handle input
- each list has an index variable that we walk along the lists
- if ingredient ID is less than lowest start bound, it's not in list, check next ingredient
- if ingredient ID is greater than or equal lowest start bound, see if it is less than corresponding end bound.
  - If less than bound, increment fresh counter, do NOT move bound index (Ranges are likely to be reused)
  - if ID greater than end bound, check next start bound

we can walk the list of ingredients by recursively following the above

In [2]:
## The recursive function to check IDs against ranges
def IDChecker(rangeIdx, ingredientsIdx, ingredientsList, ranges, freshCount):
    
    if rangeIdx >= len(ranges) or ingredientsIdx >= len(ingredientsList): # once we've exhausted either list, there are no more fresh ingredients
        return freshCount
    
    ID = ingredientsList[ingredientsIdx]
    
    if ID < ranges[rangeIdx][0]:
        # below lowest bound, not fresh
        ingredientsIdx += 1
        return IDChecker(rangeIdx, ingredientsIdx, ingredientsList, ranges, freshCount)
    
    
    if ID >= ranges[rangeIdx][0]: # compare to current lower bound
        if ID <= ranges[rangeIdx][1]: # compare to current upper bound
            # if in range, increment counter, check next ingredient
            freshCount += 1
            ingredientsIdx += 1
            return IDChecker(rangeIdx, ingredientsIdx, ingredientsList, ranges, freshCount)
        else : # above upper bound, check next range
            rangeIdx += 1
            return IDChecker(rangeIdx, ingredientsIdx, ingredientsList, ranges, freshCount)
        
## Now to read and process input
infile = open("input.txt", "r")
ranges = []
ingredients = []

for line in infile: # read in ranges
    line = line.strip()
    if line == "":
        break
    bounds = line.split("-")
    ranges.append((int(bounds[0]), int(bounds[1])))
ranges.sort(key = lambda x: (x[0], x[1])) # sort ascending by lower bound, with upper bound as tiebreaker

for line in infile: # finish reading file (ingredients)
    line = line.strip()
    ingredients.append(int(line))
ingredients.sort() # sort ingredients ascending

## Now to find the number of fresh ingredients
freshCount = IDChecker(0, 0, ingredients, ranges, 0)
print(freshCount)

577


## Part 2
- no longer checking ingredients for freshness, but just summing how many fresh ingredient ID's there are
- catch is ranges can overlap, so it's not just a straight sum of each ranges length
  - should reduce ranges down to be non-overlapping then sum the range lengths

what are the ways two ranges can be overlapping? Recall that the ranges are ordered by increasing start bound and tiebroken by increasing end bound    
There are only 9 possible states arising from `L1 (< = > L2)` and `U1 (< = >) U2`     
  - L2 can not be less than L1 due to ranges being ordered, so we have 6 valid range interactions
  - If L1=L2, U2 must > U1 due to ordering, so we have 5 valid interaction states

Case 1a: `L1 < L2` and `U1 < U2` and `L2 > U1`
<pre>
|    --------------
|                        --------------
</pre>

Case 1b: `L1 < L2` and `U1 < U2` and `L2 <= U1`
<pre>
|    --------------
|                 --------------
</pre>

Case 2: `L1 < L2` and `U1 = U2`
<pre>
|     -----------
|        --------
</pre>
Case 3: `L1 < L2` and `U1 > U2`
<pre>
|     -----------------
|        --------
</pre>
Case 4: `L1 = L2` and `U1 < U2`
<pre>
|    ----------
|    ------------------
</pre>
Case 5: `L1 = L2` and `U1 = U2`
<pre>
|     -----------
|     -----------
</pre>
Case 6: `L1 = L2` and `U1 > U2`  **Illegal state** if lower bounds are equal, then sort uses upper bound, U2 must be > U1
<pre>
|     -----------
|     --------
</pre>

Case 1a:
- disjoint ranges, keep both

Case 1b:
- extend the range, return one range [L1,U2]

Case 2, 3:
  - range 1 absorbs range 2, return only the first range
  
Case 4, 5:
- range 2 absorbs range 1, return only the second range




In [7]:
# copying same input method for ranges
infile = open("input.txt", "r")
ranges = []
freshIDCount = 0

for line in infile: # read in ranges
    line = line.strip()
    if line == "":
        break
    bounds = line.split("-")
    ranges.append((int(bounds[0]), int(bounds[1])))
ranges.sort(key = lambda x: (x[0], x[1])) # sort ascending by lower bound, with upper bound as tiebreaker

# now to reduce ranges list into non-overlapping ranges
listIndex = 0
while listIndex < len(ranges)-1:
    firstRange = ranges[listIndex]
    secondRange = ranges[listIndex+1]

    L1 = firstRange[0]
    U1 = firstRange[1]
    L2 = secondRange[0]
    U2 = secondRange[1]

    if L1 < L2 and U1 < U2 and U1 < L2: # case 1a: no overlap, keep both, check next
        listIndex += 1
        continue  
    elif L1 < L2 and U1 < U2 and U1 >= L2: # case 1b: overlap, merge ranges
        ranges.remove(secondRange) #listIndex is index of first range
        ranges[listIndex] = (L1, U2)
        continue
    elif L1 < L2 and U1 >= U2: # cases 2,3: second range within first range, remove second range
        ranges.remove(secondRange)
        continue 
    elif L1 == L2 and U1 <= U2: # cases 4,5: First range within second range, keep second
        ranges.remove(secondRange)
        ranges[listIndex] = (L2, U2)
        continue
    else:
        print("How did we get here?")
        print(f"listIndex: {listIndex}\tfirst range: {firstRange}\tsecond range: {secondRange}")


# now we can sum the range lengths
totalFreshIDs = 0
for bounds in ranges:
    rangeLength = bounds[1] - bounds[0] + 1
    totalFreshIDs += rangeLength

print(f"There a total of {totalFreshIDs} IDs belonging to fresh ranges")

There a total of 350513176552950 IDs belonging to fresh ranges


# Recap!

## Part 1
- Answer: 577
- It worked first try! I'm pretty happy I got the recursive structure down in one go.

## Part 2
- Answer: 350513176552950
- This also went first try! It seems like properly planning out everything first really does make a difference, who knew lol
  - approach was by case, which worked for a small case size of 5